In [6]:
!pip install greykite -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.5/22.5 MB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.9/499.9 kB 41.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 77.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.8/102.8 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 86.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
prophet 1.1.4 requires holidays>=0.25, but you have holidays 0.24 which is incompatible.


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
from greykite.framework.templates.autogen.forecast_config import ForecastConfig
from greykite.framework.templates.autogen.forecast_config import MetadataParam
from greykite.framework.templates.forecaster import Forecaster
from greykite.framework.templates.model_templates import ModelTemplateEnum
from greykite.framework.utils.result_summary import summarize_grid_search_results

from statsmodels.tsa.holtwinters import ExponentialSmoothing

from prophet import Prophet

import xgboost as xgb

import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import pacf

from scipy.stats import skew, kurtosis
from statsmodels.tsa.seasonal import seasonal_decompose


import numpy as np
import pandas as pd

In [61]:
def calculate_mape(true_values, predicted_values):
    epsilon = 1e-10
    abs_percentage_error = np.abs((true_values - predicted_values) / (true_values + epsilon))
    return round(np.mean(abs_percentage_error) * 100 , 4)

def forecast_greykite(df , freq ="D" ,forecast=False , forecast_period = 100):
  metadata = MetadataParam(
     time_col="date",
     value_col="value",
     freq=freq ,
     train_end_date=df['date'].iloc[-1]
  )
  forecaster = Forecaster()
  result = forecaster.run_forecast_config(
      df=df,
      config=ForecastConfig(
          model_template=ModelTemplateEnum.SILVERKITE.name,
          forecast_horizon=forecast_period,
          coverage=0.95,
          metadata_param=metadata
      )
  )

  if not forecast:
    return result.forecast.df['forecast'].tolist()[:-forecast_period]
  return result.forecast.df['forecast'].tolist()

def forecast_xgboost(df , forecast_period = 0 , date_to = None):
  df['year'] = df['date'].dt.year
  df['month'] = df['date'].dt.month
  df['day'] = df['date'].dt.day
  df['day_of_week'] = df['date'].dt.dayofweek

  X_train = df.drop(['value' , 'date'], axis=1)
  y_train = df['value']

  model = xgb.XGBRegressor(max_depth=3, min_child_weight=1, reg_alpha=0.1)
  model.fit(X_train, y_train)

  past = model.predict(X_train).tolist()

  if forecast_period and date_to:
    date_range = pd.date_range(start=date_to + pd.DateOffset(days=1), periods=forecast_period, freq='D' , normalize=True)
    future_data = pd.DataFrame(index=date_range)
    future_data['year'] = future_data.index.year
    future_data['month'] = future_data.index.month
    future_data['day'] = future_data.index.day
    future_data['day_of_week'] = future_data.index.dayofweek

    future_predictions = model.predict(future_data).tolist()
    past.extend(future_predictions)
    return past

  return past


def forecast_ets(df , forecast_period = 0 ):
  model = ExponentialSmoothing(df['value'], trend='add', seasonal='add', seasonal_periods=12)
  result = model.fit()

  past = result.fittedvalues.tolist()
  if forecast_period:
    future_data = result.forecast(steps = forecast_period)
    past.extend(future_data)
    return past

  return past

def forecast_prophet(df , forecast_period = 0 ):
  df = df.rename(columns={'date': 'ds', 'value': 'y'})

  model = Prophet()
  model.fit(df)

  future = model.make_future_dataframe(periods=forecast_period)
  forecast = model.predict(future)
  return forecast['yhat'].tolist()


def forecast_arima(df , forecast_period = 0):
  result = adfuller(df['value'])
  p_value = result[1]

  if p_value < 0.05:
      d = 0
  else:
      d = 1

  pacf_values = pacf(df['value'], method='ols')
  acf_values = pacf(df['value'], method='ols')

  for p in range(1, len(pacf_values)):
      if abs(pacf_values[p]) < 1.96 / np.sqrt(len(df)):
          break

  for q in range(1, len(acf_values)):
      if abs(acf_values[q]) < 1.96 / np.sqrt(len(df)):
          break

  model = sm.tsa.arima.ARIMA(df['value'], order=(p, d, q))
  results = model.fit()
  past = results.fittedvalues.tolist()

  if forecast_period:
    future_data = results.forecast(steps=forecast_period)
    past.extend(future_data)
    return past

  return past

def train_greykite(df, freq="D"):
    metadata = MetadataParam(
        time_col="date",
        value_col="value",
        freq=freq,
        train_end_date=df['date'].iloc[-1]
    )
    forecaster = Forecaster()
    result = forecaster.run_forecast_config(
        df=df,
        config=ForecastConfig(
            model_template=ModelTemplateEnum.SILVERKITE.name,
            metadata_param=metadata
        )
    )

    trained_values  = result.forecast.df['forecast'].tolist()[:len(df)]
    return trained_values

In [10]:
file_path = '/content/drive/MyDrive/DGdataset/Sample Time Series/daily/processed_daily.csv'
df = pd.read_csv(file_path , index_col = 0)
df.head()

,date,value
568,2021-02-01,903053.004725
569,2021-02-02,910297.753775
570,2021-02-03,912559.254575
571,2021-02-04,920697.253775
572,2021-02-05,927060.753775


In [11]:
df['date'] = pd.to_datetime(df['date'])
df.set_index(df['date'], inplace=True)
df.sort_index(inplace=True)
df.rename_axis('index',inplace=True)
df.head()

,date,value
index,,
2021-02-01,2021-02-01,903053.004725
2021-02-02,2021-02-02,910297.753775
2021-02-03,2021-02-03,912559.254575
2021-02-04,2021-02-04,920697.253775
2021-02-05,2021-02-05,927060.753775


In [12]:
df.tail()

,date,value
index,,
2023-01-28,2023-01-28,27.0
2023-01-29,2023-01-29,153181.0
2023-01-30,2023-01-30,541232.0
2023-01-31,2023-01-31,712411.0
2023-02-01,2023-02-01,1.0


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 731 entries, 2021-02-01 to 2023-02-01
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    731 non-null    datetime64[ns]
 1   value   731 non-null    float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 17.1 KB


In [67]:
import warnings
import logging
import sys


warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=FutureWarning, module="prophet")

logging.basicConfig(level=logging.WARNING)

# stdout_orig = sys.stdout
# stderr_orig = sys.stderr

# sys.stdout = open("output.log", "w")  # Replace "output.log" with the desired file name
# sys.stderr = open("error.log", "w")    # Replace "error.log" with the desired file name

In [30]:
np.random.seed(0)

date_rng = pd.date_range(start='2023-01-01', periods=100, freq='D')

# Generate synthetic data with a linear trend and autocorrelation
t = np.arange(0, 100)
trend = 0.1 * t  # Linear trend
autocorrelation_component = 0.7 * np.random.normal(0, 1, 100)  # Autocorrelation
time_series_data = 50 + trend + autocorrelation_component

data_arima = pd.DataFrame({'date': date_rng, 'value': time_series_data})
data_arima['date'] = pd.to_datetime(data_arima['date'])
data_arima.set_index(data_arima['date'], inplace=True)
data_arima.sort_index(inplace=True)
data_arima.rename_axis('index',inplace=True)
data_arima.head()

,date,value
index,,
2023-01-01,2023-01-01,51.234837
2023-01-02,2023-01-02,50.380110
2023-01-03,2023-01-03,50.885117
2023-01-04,2023-01-04,51.868625
2023-01-05,2023-01-05,51.707291


In [21]:
def generate_data(df , sample_size = 100):

  results_df = pd.DataFrame(columns =[
      'Sample Index',
      'Mean',
      'Variance',
      'Skewness',
      'Kurtosis',
      'Trend Mean',
      'Seasonal Mean',
      'Residual Mean',
      'Best Model',
  ])
  track_log = {}

  #(len(df)-sample_size)
  for start_index in range(len(df)-sample_size):
    sample_df = df.iloc[start_index:start_index+sample_size]

    true_values = sample_df['value']

    mape_prophet = calculate_mape(true_values, forecast_prophet(sample_df))
    mape_arima = calculate_mape(true_values, forecast_arima(sample_df))
    mape_ets = calculate_mape(true_values, forecast_ets(sample_df))
    mape_xgboost = calculate_mape(true_values, forecast_xgboost(sample_df))
    # mape_greykite = calculate_mape(true_values, train_greykite(sample_df))
    # mape_xgboost = float("inf")
    mape_greykite = float("inf")

    res = [
        ('Prophet', mape_prophet),
        ('ARIMA', mape_arima),
        ('ETS', mape_ets),
        ('XGBoost', mape_xgboost),
        ('Greykite', mape_greykite)
    ]

    track_log[start_index] = res
    best_model = min(res, key=lambda x: x[1])[0]

    # Calculate statistical features
    mean = sample_df['value'].mean()
    variance = sample_df['value'].var()
    skewness = skew(sample_df['value'])
    kurt = kurtosis(sample_df['value'])

    # Calculate time-based features
    result = seasonal_decompose(sample_df['value'], model='additive', extrapolate_trend='freq')
    trend_mean = result.trend.mean()
    seasonal_mean = result.seasonal.mean()
    residual_mean = result.resid.mean()

    result_dict = {
          'Sample Index': start_index,
          'Mean': mean,
          'Variance': variance,
          'Skewness': skewness,
          'Kurtosis': kurt,
          'Trend Mean': trend_mean,
          'Seasonal Mean': seasonal_mean,
          'Residual Mean': residual_mean,
          'Best Model': best_model,
      }

    results_df = results_df.append(result_dict, ignore_index=True)
    print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++" , start_index)
    print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++" , result_dict)


  return track_log , results_df

In [ ]:
track_log , results_df = generate_data(df , 25)

In [89]:
from sklearn.model_selection import TimeSeriesSplit

n_splits = 250
tscv = TimeSeriesSplit(n_splits=n_splits)

X = df.drop(['value'], axis=1)
y = df['value']

results_df = pd.DataFrame(columns =[
    'Sample Index',
    'Mean',
    'Variance',
    'Skewness',
    'Kurtosis',
    'Trend Mean',
    'Seasonal Mean',
    'Residual Mean',
    'Best Model',
])
track_log = {}

for start_index, (train_index, test_index) in enumerate(tscv.split(X)):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]

    sample_df = pd.concat([X_train, y_train], axis=1)


    mape_prophet = calculate_mape(y_test, forecast_prophet(sample_df , forecast_period = len(X_test))[-len(X_test):] )
    mape_arima = calculate_mape(y_test, forecast_arima(sample_df  , forecast_period = len(X_test))[-len(X_test):])
    mape_ets = calculate_mape(y_test, forecast_ets(sample_df , forecast_period = len(X_test))[-len(X_test):])
    mape_xgboost = calculate_mape(y_test, forecast_xgboost(sample_df ,  forecast_period = len(X_test) , date_to = X_test['date'][0])[-len(X_test):])
    # mape_greykite = calculate_mape(y_test, forecast_greykite(sample_df , forecast = True , forecast_period = len(X_test))[-len(X_test):])
    mape_greykite = float("inf")

    res = [
        ('Prophet', mape_prophet),
        ('ARIMA', mape_arima),
        ('ETS', mape_ets),
        ('XGBoost', mape_xgboost),
        ('Greykite', mape_greykite)
    ]

    track_log[start_index] = res
    best_model = min(res, key=lambda x: x[1])[0]

    # Calculate statistical features
    mean = sample_df['value'].mean()
    variance = sample_df['value'].var()
    skewness = skew(sample_df['value'])
    kurt = kurtosis(sample_df['value'])

    # Calculate time-based features
    result = seasonal_decompose(sample_df['value'], model='additive', extrapolate_trend='freq')
    trend_mean = result.trend.mean()
    seasonal_mean = result.seasonal.mean()
    residual_mean = result.resid.mean()

    result_dict = {
          'Sample Index': start_index,
          'Mean': mean,
          'Variance': variance,
          'Skewness': skewness,
          'Kurtosis': kurt,
          'Trend Mean': trend_mean,
          'Seasonal Mean': seasonal_mean,
          'Residual Mean': residual_mean,
          'Best Model': best_model,
      }

    results_df = results_df.append(result_dict, ignore_index=True)

    print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++" , res)
    print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++" , best_model , start_index , len(X_train) ,len(X_test))

results_df.to_csv('/content/drive/MyDrive/DGdataset/Sample Time Series/daily/classification_attempt_5.csv')

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpmo9k28to/j5sg0f42.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpmo9k28to/gcrtj09v.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=92920', 'data', 'file=/tmp/tmpmo9k28to/j5sg0f42.json', 'init=/tmp/tmpmo9k28to/gcrtj09v.json', 'output', 'file=/tmp/tmpmo9k28to/prophet_model71wfaeai/prophet_model-20230930121026.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
12:10:26 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
12:10:26 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
INFO:prophet:Disabling yearly seasonalit

+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++ [('Prophet', 696093.283), ('ARIMA', 827922.8368), ('ETS', 722630.8563), ('XGBoost', 847028.6013), ('Greykite', inf)]
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++ Prophet 0 231 2


INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpmo9k28to/sdggbeeu.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpmo9k28to/_rgv8k2s.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=10174', 'data', 'file=/tmp/tmpmo9k28to/sdggbeeu.json', 'init=/tmp/tmpmo9k28to/_rgv8k2s.json', 'output', 'file=/tmp/tmpmo9k28to/prophet_modelfg01ilk5/prophet_model-20230930121029.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
12:10:29 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
12:10:29 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++ [('Prophet', 265738.2675), ('ARIMA', 264195.2143), ('ETS', 221007.36), ('XGBoost', 99254.8593), ('Greykite', inf)]
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++ XGBoost 1 233 2


INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpmo9k28to/oich7ugc.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpmo9k28to/94r5o_mm.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=33167', 'data', 'file=/tmp/tmpmo9k28to/oich7ugc.json', 'init=/tmp/tmpmo9k28to/94r5o_mm.json', 'output', 'file=/tmp/tmpmo9k28to/prophet_modelnofq3g7j/prophet_model-20230930121030.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
12:10:30 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
12:10:30 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


...

+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++ [('Prophet', 518177.2592), ('ARIMA', 519744.5857), ('ETS', 759066.3735), ('XGBoost', 378315.65), ('Greykite', inf)]
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++ XGBoost 247 725 2


INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpmo9k28to/3mwhesss.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpmo9k28to/1pxdzpw6.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=14491', 'data', 'file=/tmp/tmpmo9k28to/3mwhesss.json', 'init=/tmp/tmpmo9k28to/1pxdzpw6.json', 'output', 'file=/tmp/tmpmo9k28to/prophet_modelqehf2tey/prophet_model-20230930123141.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
12:31:41 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing


+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++ [('Prophet', 59.1417), ('ARIMA', 51.358), ('ETS', 69.9885), ('XGBoost', 55.1817), ('Greykite', inf)]
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++ ARIMA 248 727 2


12:31:41 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++ [('Prophet', 13626138.0735), ('ARIMA', 23169176.7812), ('ETS', 17005251.3027), ('XGBoost', 14811983.8858), ('Greykite', inf)]
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++ Prophet 249 729 2


In [90]:
results_df.head()

,Sample Index,Mean,Variance,Skewness,Kurtosis,Trend Mean,Seasonal Mean,Residual Mean,Best Model
0,0,1.038035e+06,5.465685e+10,0.894983,1.447480,1.037771e+06,1.889859e-12,263.922760,Prophet
1,1,1.029126e+06,6.339224e+10,0.211591,2.800042,1.033529e+06,-7.179134e+02,-3685.880673,XGBoost
2,2,1.020369e+06,7.182116e+10,-0.212454,3.310216,1.024340e+06,-1.408817e+03,-2562.343889,XGBoost
3,3,1.011759e+06,7.996001e+10,-0.486683,3.425143,1.012884e+06,-8.785560e+02,-246.044085,ARIMA
4,4,1.003293e+06,8.781727e+10,-0.668460,3.346775,1.000470e+06,-2.136987e+02,3036.506817,ETS


In [91]:
track_log

{0: [('Prophet', 696093.283),
  ('ARIMA', 827922.8368),
  ('ETS', 722630.8563),
  ('XGBoost', 847028.6013),
  ('Greykite', inf)],
 1: [('Prophet', 265738.2675),
  ('ARIMA', 264195.2143),
  ('ETS', 221007.36),
  ('XGBoost', 99254.8593),
  ('Greykite', inf)],
 2: [('Prophet', 890080.7727),
  ('ARIMA', 478565.3411),
  ('ETS', 366005.5838),
  ('XGBoost', 350735.4762),
  ('Greykite', inf)],
 3: [('Prophet', 1259697.8257),
  ('ARIMA', 107993.838),
  ('ETS', 123869.9662),
  ('XGBoost', 198203.5527),
  ('Greykite', inf)],
 4: [('Prophet', 874453.0562),
  ('ARIMA', 323458.83),
  ('ETS', 36198.1454),
  ('XGBoost', 180111.1157),
  ('Greykite', inf)],
 5: [('Prophet', 271208.2009),
  ('ARIMA', 69448.4532),
  ('ETS', 104762.6019),
  ('XGBoost', 568796.4267),
  ('Greykite', inf)],
 6: [('Prophet', 484900.5595),
  ('ARIMA', 8070.0692),
  ('ETS', 35744.4329),
  ('XGBoost', 175152.6264),
  ('Greykite', inf)],
 7: [('Prophet', 192801.7352),
  ('ARIMA', 34232.0615),
  ('ETS', 37364.0465),
  ('XGBoost', 4

In [92]:
results_df['Best Model'].value_counts()

ARIMA      169
XGBoost     51
ETS         20
Prophet     10
Name: Best Model, dtype: int64

In [ ]:
results_df

In [21]:
track_log

{0: [('Prophet', 1.3564),
  ('ARIMA', 9.3049),
  ('ETS', 12.9386),
  ('XGBoost', inf),
  ('Greykite', inf)],
 1: [('Prophet', 1.3251),
  ('ARIMA', 7.2567),
  ('ETS', 13.1763),
  ('XGBoost', inf),
  ('Greykite', inf)],
 2: [('Prophet', 1.3334),
  ('ARIMA', 7.5155),
  ('ETS', 13.0577),
  ('XGBoost', inf),
  ('Greykite', inf)],
 3: [('Prophet', 1.3306),
  ('ARIMA', 7.6489),
  ('ETS', 13.3288),
  ('XGBoost', inf),
  ('Greykite', inf)],
 4: [('Prophet', 1.3369),
  ('ARIMA', 7.2046),
  ('ETS', 13.1435),
  ('XGBoost', inf),
  ('Greykite', inf)]}